# IBM Cognos Analytics -- IBM Process Mining
## Business Intelligence meets Business Process Intelligence


## Collect business performance data for each case
Theoritically, we don't yet have process mining data, and we should collect the bank account closure KPIs from the bank's datalake.
- a CSV table that lists each case ID with its start date and leadtime in milliseconds
- a CSV table that lists each case ID with some business data: CLOSURE_TYPE and CLOSURE_REASON

These 2 tables are initially used in Cognos Analytics to display a dashboard with key performance indicators:
- Number of requests in 2 years
- Average leadtime compared with the EU requirement that is 12 days
- Number of requests per type
- Evolution of number of requests per day with predictions

Since we actually have process mining, we can create these two files using the following code:

In [ ]:
import IPMClient as ipm
import pandas as pd
import json
from datetime import datetime as dt

ipmConfigFilename = './IPMConfig.json'
with open(ipmConfigFilename, 'r') as file:
    ipmConfig = json.load(file)  

ipmClient = ipm.Client(ipmConfig['url'], ipmConfig['userid'], ipmConfig['apikey'])
ipmProject = ipmClient.getProjectByName('Bank Account Closure')
# Apply the filter that exclude running cases   
templates = ipmProject.retrieveTemplates()
templateId = 0
for template in templates:
    if template['name'] == 'complete cases':
        templateId = template['templateId']
        break;
templateId

# FIRST CSV
query = "SELECT CASEID, MIN(starttime), leadtime FROM EVENTLOG GROUP BY CASEID"
headers = ipmClient.getHeaders()
headers['content-type'] = 'application/x-www-form-urlencoded'
data = "params={'templateId': %s, 'query': '%s'}" % (templateId, query)

res = ipmProject.sendPostRequest(
            url=f"{ipmProject.getURL()}/analytics/integration/{ipmProject.key}/query",
            verify=ipmProject.verify,
            params={'org' : ipmProject.orgkey},
            headers=headers,
            data=data,
            files=None,
            functionName='retrieve from SQL'
        )
df = pd.DataFrame(res)
df.columns = ['caseid', 'startdate', 'leadtime']
df.reindex()
df['startdate']=df['startdate'].apply(lambda x: dt.fromtimestamp(x[0]/1000))
# dates grouped by week
df['startdate']=df['startdate'].apply(lambda x: x.replace(day=1, hour=0, minute=0, second=0))

df.columns = ['caseid', 'startdate', 'leadtime']
df.reindex()
df['startdate']=df['startdate'].apply(lambda x: dt.fromtimestamp(x[0]/1000))
# dates grouped by week
df['startdate']=df['startdate'].apply(lambda x: x.replace(day=1, hour=0, minute=0, second=0))
df.to_csv('completedCasesDates.csv', index=None)

# SECOND CSV
query = "SELECT CASEID, CLOSURE_TYPE, CLOSURE_REASON FROM EVENTLOG GROUP BY CASEID"
headers = ipmClient.getHeaders()
headers['content-type'] = 'application/x-www-form-urlencoded'
data = "params={'templateId': %s, 'query': '%s'}" % (templateId, query)
res = ipmProject.sendPostRequest(
            url=f"{ipmProject.getURL()}/analytics/integration/{ipmProject.key}/query",
            verify=ipmProject.verify,
            params={'org' : ipmProject.orgkey},
            headers=headers,
            data=data,
            files=None,
            functionName='retrieve from SQL'
        )
df = pd.DataFrame(res)
df.columns = ['caseid', 'closure_type', 'closure_reason']
df.reindex()
df.to_csv('completedCasesBizData.csv', index=None)

## Load these 2 CSV files into Cognos Analytics
Within IBM Cognos Analytics, you can import these 2 files into your content.

That can also be done with the Cognos REST API with the code below.

In [ ]:
import CognosAnalyticsClient as cog
import json

cognosConfigFilename = './CognosAnalytics.json'
with open(cognosConfigFilename, 'r') as file:
    cognosConfig = json.load(file)


cognosCredentials =  cog.cognosCreateCredentials(cognosConfig)    
auth = cog.cognosCreateSession(cognosConfig['url'], credentials=cognosCredentials)
cog.cognosUploadFile(cognosConfig['url'], auth['authkey'], auth['authvalue'], filename='completedCasesDates.csv', append=False, silent=False)
cog.cognosUploadFile(cognosConfig['url'], auth['authkey'], auth['authvalue'], filename='completedCasesBizData.csv', append=False, silent=False)

## Create the data module in Cognos
We create a data module that includes these 2 files.

In this data module we:
- Join the 2 tables using CASEID
- Create a calculation field that transforms the leadtime from milliseconds to days = leadtime/(24*3600000)
- Create a calculation field that stores 12 days goal from the EU.
- Format the columns nicely

## Create a Business Performance dashboard
From the data module, we mostly use the Cognos Assistant to create a dashboard like this:

<img src="./images/business_performance.jpg " width="500" />
## Business Intelligence Meets Business Process Intelligence

The performance is not good: 20 days in average instead of 12 days, and the prediction from Cognos shows an increase.

What can we do to close bank account closure in less then 12 days?

Should we just interview client managers and process owners?

Do we have visibility and details about the business process that supports Bank Account Closure?

Let's use IBM Process Mining to clearly understand what happends, and to make factual decisions.

## IBM Process Mining demo using the Bank Account Closure Project

Standard Bank Account Closure demonstration showing frequencies, durations, costs, reworks, variants, compliance, bpmn, and a few dashboards

## Enriching Cognos Analytics Dashboards with Process Mining data

We now want to share process mining data to a wider audience that is currently Cognos Analytics:
- Actions tab: the list of the cases that require an action since they are blocked in a process step for more than a certain time
- Process Steps Statistics: The management wants to have statistics of waiting time between each step to fix new goals

The code below queries IBM Process Mining to create the CSV files that will be uploaded to IBM Cognos Analytics


In [ ]:
import IPMClient as ipm
import pandas as pd
import json

ipmConfigFilename = './IPMConfig.json'
with open(ipmConfigFilename, 'r') as file:
    ipmConfig = json.load(file)  

ipmClient = ipm.Client(ipmConfig['url'], ipmConfig['userid'], ipmConfig['apikey'])
proj = ipmClient.getProjectByName('Bank Account Closure')

# CSV for Actions. The data is in a table widget in 'Clean The Pipe' dashboard
dashboard = proj.getDashboardByName('Clean The Pipe')
widgets = dashboard.getWidgets()
df = pd.DataFrame(widgets[0].retrieveValues())
df.to_csv('blocked_at_pending_liquidation_request.csv', index=None)

# CSV for Transition Statistics
stats = proj.retrieveModelStatistics()
transitionStats = proj.getTransitionStatistics(stats)
transitionStats_df = pd.json_normalize(transitionStats)
transitionStats_df.keys()
transitionStats_df=transitionStats_df[['sourceActivity', 'targetActivity', 'statistics.frequency', 'statistics.avgDuration', 'statistics.medianDuration',
                                       'statistics.minDuration','statistics.maxDuration', 'statistics.parallelFrequency','statistics.caseRepetition','statistics.avgRepetition']]
transitionStats_df.rename({'statistics.frequency':'frequency', 'statistics.avgDuration':'avgDuration', 'statistics.medianDuration':'medianDuration',
                                       'statistics.minDuration':'minDuration','statistics.maxDuration':'maxDuration', 
                                       'statistics.parallelFrequency':'parallelFrequency','statistics.caseRepetition':'caseRepetition',
                                       'statistics.avgRepetition':'avgRepetition'}, axis='columns', inplace=True)
transitionStats_df.to_csv('transitionStats.csv', index=None)


## Upload Process Mining data to Cognos Analytics

We now want to upload these 2 files to IBM Cognos Analytics, using the REST API

In [ ]:
import CognosAnalyticsClient as cog
import json

cognosConfigFilename = './CognosAnalytics.json'
with open(cognosConfigFilename, 'r') as file:
    cognosConfig = json.load(file)


cognosCredentials =  cog.cognosCreateCredentials(cognosConfig)    
auth = cog.cognosCreateSession(cognosConfig['url'], credentials=cognosCredentials)
cog.cognosUploadFile(cognosConfig['url'], auth['authkey'], auth['authvalue'], filename='blocked_at_pending_liquidation_request copy.csv', append=False, silent=False)
#cog.cognosUploadFile(cognosConfig['url'], auth['authkey'], auth['authvalue'], filename='transitionStats.csv', append=False, silent=False)